In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from collections import OrderedDict
from gensim.models import word2vec
import re
import tool
import pickle
import time
import MeCab
import tensorflow as tf

In [2]:
All_df = pd.read_pickle("../data/all_mecab.pickle")
All_df

,sen,sen_pre,say_id,reply_id,group_id,name,body,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,"[よろしく, お願い, し, ます, ！, EOS]",[EOS],1,-1,1234568,まこぴす,よろしくお願いします！,5,5,21,21,31,31,40,40,50,50
1,"[よろしく, お願い, し, ます, EOS]","[よろしく, お願い, し, ます, ！, EOS]",31,-1,1234568,哲,よろしくお願いします,5,5,21,21,31,31,40,40,50,50
2,"[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...","[よろしく, お願い, し, ます, EOS]",70,-1,1234568,仙波,名前なのが恥ずかしいです…\nよろしくお願いします！,5,5,21,21,31,31,40,40,50,50
3,"[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...",119,-1,1234568,まこぴす,早速課題やっちゃいましょう！,14,14,21,21,31,31,40,40,52,52
4,"[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...","[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]",163,-1,1234568,仙波,やっちゃいましょう\nmoodleはゴミです！,1,2,22,22,32,32,41,40,50,50
5,"[使い, にくい, です, 笑, EOS]","[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...",194,-1,1234568,まこぴす,使いにくいです(笑),2,2,22,22,32,32,41,40,50,50
6,"[同意, です, EOS]","[使い, にくい, です, 笑, EOS]",302,-1,1234568,哲,同意です,1,1,22,22,32,32,42,42,50,50
7,"[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...","[同意, です, EOS]",309,-1,1234568,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,2,2,23,23,32,32,41,41,50,50
8,"[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...","[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...",385,-1,1234568,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,2,2,25,25,32,32,41,41,50,50
9,"[自分, の, 意見, です, ！, 笑, EOS]","[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...",426,-1,1234568,まこぴす,自分の意見です！(笑),2,2,21,22,31,32,40,41,50,50


# senとsen_preの単語をIDに変換し、新たな列としてAll_dfに追加する

In [3]:
sen = All_df['sen'].values
sen_pre = All_df['sen_pre'].values

# 単語辞書の作成
wd_set = Counter([x for s in (sen + sen_pre) for x in s])
wd_ary = np.array(list(wd_set.keys()))
wd_cnt = np.array(list(wd_set.values()))

# 出現頻度順にソート
wd_ary = wd_ary[np.argsort(wd_cnt)[::-1]]
wd_cnt.sort()
wd_cnt = wd_cnt[::-1]

# 単語ID辞書の作成
wd_id = {wd: i for i, wd in enumerate(wd_ary)}

print(wd_set.most_common(10))
for i in range(10):
    print("単語:",
          list(wd_ary)[i], "| 出現数:",
          list(wd_cnt)[i], "| ID:", wd_id[list(wd_ary)[i]])

# 出現数CUT_OFF以下の単語のIDを統一
CUT_OFF = 2
print("words kinds:", len(wd_cnt), "words>=" + str(CUT_OFF) + ":",
      np.sum(wd_cnt >= CUT_OFF))
other_id = np.sum(wd_cnt >= CUT_OFF)
wd_id.update({wd: other_id for wd in wd_ary[wd_cnt < CUT_OFF]})

# senとsen_preの単語をIDに変換
sen_id = []
sen_pre_id = []
for s, s_pre in zip(sen, sen_pre):
    sen_id.append([str(wd_id[wd]) for wd in s])
    sen_pre_id.append([str(wd_id[wd]) for wd in s_pre])

# 新し列としてAll_dfに追加
All_df.insert(loc=0, column='sen_id', value=sen_id)
All_df.insert(loc=1, column='sen_pre_id', value=sen_pre_id)

[('EOS', 31346), ('の', 10568), ('て', 7406), ('です', 7391), ('ます', 7363), ('か', 7284), ('、', 6958), ('に', 6914), ('が', 6893), ('は', 6793)]
単語: EOS | 出現数: 31346 | ID: 0
単語: の | 出現数: 10568 | ID: 1
単語: て | 出現数: 7406 | ID: 2
単語: です | 出現数: 7391 | ID: 3
単語: ます | 出現数: 7363 | ID: 4
単語: か | 出現数: 7284 | ID: 5
単語: 、 | 出現数: 6958 | ID: 6
単語: に | 出現数: 6914 | ID: 7
単語: が | 出現数: 6893 | ID: 8
単語: は | 出現数: 6793 | ID: 9
words kinds: 7070 words>=2: 6686


In [4]:
All_df

,sen_id,sen_pre_id,sen,sen_pre,say_id,reply_id,group_id,name,body,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,"[39, 35, 12, 4, 18, 0]",[0],"[よろしく, お願い, し, ます, ！, EOS]",[EOS],1,-1,1234568,まこぴす,よろしくお願いします！,5,5,21,21,31,31,40,40,50,50
1,"[39, 35, 12, 4, 0]","[39, 35, 12, 4, 18, 0]","[よろしく, お願い, し, ます, EOS]","[よろしく, お願い, し, ます, ！, EOS]",31,-1,1234568,哲,よろしくお願いします,5,5,21,21,31,31,40,40,50,50
2,"[421, 19, 1, 8, 2228, 3, 0, 39, 35, 12, 4, 18, 0]","[39, 35, 12, 4, 0]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...","[よろしく, お願い, し, ます, EOS]",70,-1,1234568,仙波,名前なのが恥ずかしいです…\nよろしくお願いします！,5,5,21,21,31,31,40,40,50,50
3,"[1888, 45, 143, 296, 56, 54, 27, 18, 0]","[421, 19, 1, 8, 2228, 3, 0, 39, 35, 12, 4, 18, 0]","[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...",119,-1,1234568,まこぴす,早速課題やっちゃいましょう！,14,14,21,21,31,31,40,40,52,52
4,"[143, 296, 56, 54, 27, 0, 663, 9, 2341, 3, 18, 0]","[1888, 45, 143, 296, 56, 54, 27, 18, 0]","[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...","[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]",163,-1,1234568,仙波,やっちゃいましょう\nmoodleはゴミです！,1,2,22,22,32,32,41,40,50,50
5,"[197, 907, 3, 112, 0]","[143, 296, 56, 54, 27, 0, 663, 9, 2341, 3, 18, 0]","[使い, にくい, です, 笑, EOS]","[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...",194,-1,1234568,まこぴす,使いにくいです(笑),2,2,22,22,32,32,41,40,50,50
6,"[1759, 3, 0]","[197, 907, 3, 112, 0]","[同意, です, EOS]","[使い, にくい, です, 笑, EOS]",302,-1,1234568,哲,同意です,1,1,22,22,32,32,42,42,50,50
7,"[1669, 1400, 6500, 739, 12, 2, 56, 10, 1, 3, 8...","[1759, 3, 0]","[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...","[同意, です, EOS]",309,-1,1234568,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,2,2,23,23,32,32,41,41,50,50
8,"[44, 431, 118, 168, 1862, 1, 181, 9, 5281, 2, ...","[1669, 1400, 6500, 739, 12, 2, 56, 10, 1, 3, 8...","[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...","[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...",385,-1,1234568,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,2,2,25,25,32,32,41,41,50,50
9,"[68, 1, 132, 3, 18, 112, 0]","[44, 431, 118, 168, 1862, 1, 181, 9, 5281, 2, ...","[自分, の, 意見, です, ！, 笑, EOS]","[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...",426,-1,1234568,まこぴす,自分の意見です！(笑),2,2,21,22,31,32,40,41,50,50


# 必要なデータを取り出す

In [5]:
# データの整理（一致、重複）
print("全データ：", All_df.shape)
drop_All_df = All_df.drop_duplicates(subset=['body', 'name']).reset_index(
    drop=True)
print("重複投稿を排除したデータ：", drop_All_df.shape)

# 16types
t16_All_df = drop_All_df
print("16types有効データ(重複削除)：", t16_All_df.shape)
valid = t16_All_df.shape[0]
t16_All_df = t16_All_df[t16_All_df['16types_a'] == t16_All_df[
    '16types_b']].reset_index(drop=True)
print("16types一致データ(重複削除)：", t16_All_df.shape)
print("16types一致率：", valid, "/", t16_All_df.shape[0], "=",
      t16_All_df.shape[0] / valid)
print(Counter(t16_All_df['16types_a']), "\n")

全データ： (12012, 19)
重複投稿を排除したデータ： (11357, 19)
16types有効データ(重複削除)： (11357, 19)
16types一致データ(重複削除)： (7926, 19)
16types一致率： 11357 / 7926 = 0.6978955710134719
Counter({1: 1867, 2: 1325, 5: 1046, 3: 900, 4: 885, 6: 779, 8: 236, 7: 227, 14: 179, 9: 141, 11: 108, 15: 102, 12: 63, 13: 30, 0: 21, 10: 17}) 



In [45]:
sen_id = t16_All_df['sen_id'].values
sen_labels = t16_All_df['16types_a'].values
sen_labels = np.array(sen_labels, dtype=np.int32)
max_len = max([len(s) for s in sen_id])
sen_fv = []
for s in sen_id:
    s_pad = np.pad(
        list(map(int, s)), (0, max_len - len(s)),
        'constant',
        constant_values=(0, 0))
    sen_fv.append(s_pad)
sen_fv = np.array(sen_fv)

print(max_len)
print(sen_fv.shape)
print(sen_labels.shape)
print(sen_fv[0])
print(sen_labels[0])
data = tool.read_data_sets(sen_fv,sen_labels,one_hot=True,num_classes=16)

292
(7926, 292)
(7926,)
[39 35 12  4 18  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]
5


In [54]:
seq_length = data.train.vectors.shape[1]
num_classes = data.train.labels.shape[1]
print(sequence_length)
print(num_classes)


class TextCNN(object):
    def __init__(self,
                 seq_length,
                 num_classes,
                 vocab_size,
                 embedding_size,
                 filter_sizes,
                 num_filters,
                 l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, seq_length])
        self.input_y = tf.placeholder(tf.int32, [None, num_classes])
        self.dropout_keep_prob = tf.placeholder(tf.float32)

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.name_scope("embedding"):

            # 用来保存词向量矩阵W,形状为二维(字典长度*单个词向量长度)
            self.W = tf.Variable([vocab_size, embedding_size], -1.0, 1.0)
            # 用来保存转化成词向量后的输入数据,形状为三维(输入数据个数*句子中单词个数*单个词向量长度)
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            # 增加一个channel维度,符合图像的习惯
            self.embedded_chars_expanded = tf.expand_dims(
                self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_sizes in enumerate(
                filter_sizes):  #三种卷积核(0,3),(1,4),(2,5)
            with tf.name_scope("conv-maxpool-%s" % filter_sizes):
                # Convolution Layer

                # 卷积核参数(高*宽*通道*卷积个数)
                filter_shape = [filter_sizes, embedding_size, 1, num_filters]
                # 存放权重值
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1))
                # 存放偏置值
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]))
                # 卷积操作
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID")

                # Apply nonlinearity
                # relu操作
                h = tf.nn.relu(tf.nn.bias_add(conv, b))

                # Maxpooling over the outputs
                # maxpooling操作
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID')

                # 三种卷积核,append三次,形状为(输入数据个数*1*1*卷积核个数)
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        # 每种卷积核个数与卷积核种类的积
        num_filters_total = num_filters * len(filter_sizes)
        # 将outpus在第4个维度上拼接，如本来是128*1*1*64的结果3个，拼接后为128*1*1*192
        self.h_pool = tf.concat(pooled_outputs, 3)
        # 结果reshape为128*192
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            # dropout操作,输出为dropout后的128*192
            self.h_drop = tf.nn.dropout(self.h_pool_flat,
                                        self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]))
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b)
            self.predictions = tf.argmax(self.scores, 1)

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(
                logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions,
                                           tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(
                tf.cast(correct_predictions, "float"))

292
16
